<a href="https://colab.research.google.com/github/profteachkids/CHE2064_Spring2022/blob/main/DistillationNRTL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget -N -q https://raw.githubusercontent.com/profteachkids/chetools/main/tools/che.py
!wget -N -q https://raw.githubusercontent.com/profteachkids/chetools/main/tools/FlattenWrap.ipynb
!pip install importnb

In [2]:
from importnb import Notebook
with Notebook(): 
    from FlattenWrap import DotDict, Range, RangeArray, Comp, CompArray, dtox

import che
import numpy as np
from scipy.optimize import root

In [3]:
p=che.Props(['Ethanol','Water'])

In [117]:
d=DotDict()

d.P = 1e5
d.N = 5
d.NC = p.N_comps
d.F= 100.
d.Fz = np.array([0.2,0.8])
d.FT =320.
d.D = 0.25*d.F
d.B = d.F - d.D

d.F_stage = d.N // 2

d.reflux_ratio = 10.


In [118]:
def boiling_points(P):
    return root(lambda T: np.diagonal(p.Pvap(T)) - P, p.Tb).x

In [119]:
d.bp = boiling_points(d.P)
bp_scaled = (d.bp - np.mean(d.bp))/1.2 + np.mean(d.bp)
bp_minmax = np.min(d.bp), np.max(d.bp)

# Bug in root solver: convergence failure if T_guess is exactly linearly spaced
d.T = RangeArray(np.round(np.linspace(*bp_scaled,d.N)), *bp_minmax)
d.TC = Range(bp_scaled[-1], *bp_minmax)

Lguess = np.repeat(d.D*(d.reflux_ratio),d.N)
Lguess[d.F_stage:]+=d.F

d.L = RangeArray(Lguess, 0., 2*d.F*d.reflux_ratio)
d.V = RangeArray(np.repeat(d.D*(d.reflux_ratio+1),d.N), 0., 2*d.F*d.reflux_ratio)

QBGuess = d.D * d.reflux_ratio* np.mean(p.HvapNB)
d.QB = Range(QBGuess, 0., 2*QBGuess )
d.QC = Range(QBGuess, 0., 2*QBGuess)

d.Lx = CompArray(np.tile(d.Fz,(d.N,1)))
d.Vy = CompArray(np.tile(d.Fz,(d.N,1)))



In [120]:
wrap, x, d2, xtod, xtodunk  = dtox(d)

In [121]:
r = DotDict()

r.MB = np.zeros((d.N,d.NC))
zeros = np.zeros((1,p.N_comps))
def eqs(d):
    r.EQUIL = d.Lx * p.NRTL_gamma(d.Lx,d.T)* p.Pvap(d.T) - d.Vy*d.P


    r.Lin = np.r_[(d.L[0]*d.Vy[0])[None,:], d.L[1:,None]*d.Lx[:-1]]

    r.Lout = np.r_[d.L[1:,None]*d.Lx[:-1], (d.B*d.Lx[-1])[None,:]]

    r.Vin = np.r_[d.V[1:,None]*d.Vy[1:], zeros]
    r.Vout = d.V[:,None]*d.Vy

    r.MB = r.Lin - r.Lout + r.Vin - r.Vout

    r.MB[d.F_stage-1] += d.F*d.Fz

    r.TCBP = np.atleast_1d(np.sum(d.Vy[0]* p.NRTL_gamma(d.Vy[0],d.TC) * p.Pvap(d.TC)) - d.P)

    r.T = np.insert(d.T,0,d.TC)
    r.EB = np.array(p.Hl(r.Lin, r.T[:-1]) + p.Hv(r.Vin, r.T[1:]) - p.Hl(r.Lout, r.T[1:]) - p.Hv(r.Vout, r.T[1:]))
    r.EB[d.F_stage-1] += np.array(p.Hl(d.F*d.Fz, d.FT))
    r.EB[-1]+=d.QB

    r.CONDENSER = np.atleast_1d(p.Hv(r.Vout[0],r.T[1]) - p.Hl(r.Vout[0], r.T[0]) - d.QC)

    r.RF = np.atleast_1d((d.V[0]-d.D)/d.D) - d.reflux_ratio
    return np.concatenate([np.ravel(r.EQUIL), np.ravel(r.MB), np.ravel(r.EB), r.TCBP, r.RF, r.CONDENSER]), r


In [122]:
eqs_wrapped = wrap(eqs)

In [123]:
eqs(d2)

(array([   28260.66411978,   -38557.22261603,    35975.55556413,
          -31439.28632752,    47001.12752078,   -21130.74535836,
           57035.003219  ,   -11625.43303876,    71246.8771669 ,
            2017.08176322,        0.        ,        0.        ,
               0.        ,        0.        ,        0.        ,
               0.        ,        0.        ,        0.        ,
               0.        ,        0.        ,   402568.77421135,
         -408847.08628159,  -156128.98719361,  -125812.5516936 ,
        -1268325.64603498,    73111.08838542,        0.        ,
          927427.86421982]),
 {'CONDENSER': array([927427.86421982]),
  'EB': array([  402568.77421135,  -408847.08628159,  -156128.98719361,
          -125812.5516936 , -1268325.64603498]),
  'EQUIL': DeviceArray([[ 28260.66411978, -38557.22261603],
               [ 35975.55556413, -31439.28632752],
               [ 47001.12752078, -21130.74535836],
               [ 57035.003219  , -11625.43303876],
           

In [ ]:
sol=root(eqs_wrapped, x, )
print(sol)
solx=sol.x

In [125]:
res=xtod(sol.x,d2)
print(res.Lx)
print(res.Vy)

[[0.64523761 0.35476239]
 [0.49593656 0.50406344]
 [0.41502449 0.58497551]
 [0.17359086 0.82640914]
 [0.02595959 0.97404041]]
[[0.72212123 0.27787877]
 [0.65225893 0.34774107]
 [0.62144007 0.37855993]
 [0.51955532 0.48044468]
 [0.21385867 0.78614133]]


In [ ]:
eqs(xtod(sol.x,d2))